In [1]:
import os
import numpy as np
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import random

#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)

In [2]:
if not (os.path.exists('./Data/preprocessed_user_item_rating.csv')):
  raise ValueError('El archivo preprocessed_user_item_rating.csv no fue encontrado en el path')
else:
  print("El archivo ha sido cargado")

El archivo ha sido cargado


In [3]:
ratings=pd.read_csv('./Data/preprocessed_user_item_rating.csv', sep = ',', header=0, names = [ '', 'userid', 'artist-name', 'count', 'max', 'rating' ] )
ratings = ratings.loc[:,['userid', 'artist-name','rating']]
ratings

,userid,artist-name,rating
0,user_001000,Wilco,10.000000
1,user_001000,Radiohead,9.758242
2,user_001000,Animal Collective,9.494505
3,user_001000,Girl Talk,9.164835
4,user_001000,Aesop Rock,8.417582
...,...,...,...
894494,user_000001,曽我部恵一,0.011696
894495,user_000001,畠山美由紀,0.011696
894496,user_000001,細野晴臣,0.011696
894497,user_000001,青柳拓次,0.011696


In [10]:
reduced_ratings = ratings.head(1000)

# Creacion sistema de recomendacion

In [11]:
reader = Reader( rating_scale = ( 1, 10 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( reduced_ratings[ [ 'userid', 'artist-name', 'rating' ] ], reader )

In [12]:
#Se crea el dataset para modelo 
rating_data=surprise_dataset.build_full_trainset()
# Se crea dataset de "prueba" con las entradas faltantes para generar las predicciones
test=rating_data.build_anti_testset()

### Modelo basado en distancias coseno

In [13]:
sim_options = {'name': 'cosine',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=20, min_k=10, sim_options=sim_options)

### Modelo basado en índice de Jaccard

### Modelo basado en correlación de Pearson

In [ ]:
sim_options = {'name': 'pearson_baseline',
               'user_based': False,
               'shrinkage': 0  # no shrinkage
               }
algo = KNNBasic(sim_options=sim_options)

## Recomendaciones

In [14]:
algo.fit(rating_data)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:
predictions=algo.test(test)

In [16]:
#Predicciones para usuario user_001000
user_predictions=list(filter(lambda x: x[0]=='user_001000',predictions))

In [17]:
#Ordenamos de mayor a menor estimación de relevancia
user_predictions.sort(key=lambda x : x.est, reverse=True)

In [18]:
#tomamos las 10 primeras predicciones
user_predictions=user_predictions[0:10]

In [19]:
#Se convierte a dataframe
labels = ['artist', 'estimation']
df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_predictions)), columns=labels)

### Recomendaciones usuario 001000

In [20]:
df_predictions.loc[:,['artist']]

,artist
0,雅-Miyavi-
1,Panic At The Disco
2,Arctic Monkeys
3,Glen Hansard & Markéta Irglová
4,František Černý A Karel Plíhal
5,Godsmack
6,Linkin Park
7,Slipknot
8,The Chemical Brothers
9,Alanis Morissette


In [21]:
accuracy.rmse( user_predictions, verbose = True )

RMSE: 2.6379


2.637945018303915